In [0]:
spark

In [0]:
storage_account = "sample"
application_id = "sample"
directory_id = "sample"

spark.conf.set(f"fs.azure.account.auth.type.{storage_account}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{storage_account}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{storage_account}.dfs.core.windows.net", application_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{storage_account}.dfs.core.windows.net", "sample")
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{storage_account}.dfs.core.windows.net", f"https://login.microsoftonline.com/{directory_id}/oauth2/token")

In [0]:
customer_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("abfss://olistdata@olistdatastorageacthyma.dfs.core.windows.net/bronze/olist_customers_dataset.csv")

# display(customer_df)

In [0]:
geolocation_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("abfss://olistdata@olistdatastorageacthyma.dfs.core.windows.net/bronze//olist_geolocation_dataset.csv")

# display(geolocation_df)


In [0]:
items_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("abfss://olistdata@olistdatastorageacthyma.dfs.core.windows.net/bronze//olist_order_items_dataset.csv")



payments_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("abfss://olistdata@olistdatastorageacthyma.dfs.core.windows.net/bronze//olist_order_payments_dataset.csv")



reviews_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("abfss://olistdata@olistdatastorageacthyma.dfs.core.windows.net/bronze//olist_order_reviews_dataset.csv")


orders_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("abfss://olistdata@olistdatastorageacthyma.dfs.core.windows.net/bronze//olist_orders_dataset.csv")

products_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("abfss://olistdata@olistdatastorageacthyma.dfs.core.windows.net/bronze/olist_products_dataset.csv")

sellers_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("abfss://olistdata@olistdatastorageacthyma.dfs.core.windows.net/bronze//olist_sellers_dataset.csv")



In [0]:
!pip install pymongo

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


Reading Data From PyMongo

In [0]:
#Fetching data from MangoDB
from pymongo import MongoClient
import pandas as pd


In [0]:
# importing module
from pymongo import MongoClient

hostname = "xdtqr.h.filess.io"
database = "olistDataNoSQL_teacherbox"
port = "61004"
username = "olistDataNoSQL_teacherbox"
password = "fcec05230340cf66a26f410a35c9fdaae8852143"

uri = "mongodb://" + username + ":" + password + "@" + hostname + ":" + port + "/" + database

# Connect with the portnumber and host
client = MongoClient(uri)

# Access database
mydatabase = client[database]
mydatabase

Database(MongoClient(host=['xdtqr.h.filess.io:61004'], document_class=dict, tz_aware=False, connect=True), 'olistDataNoSQL_teacherbox')

In [0]:
collection=mydatabase['product_category']

mongo_data=pd.DataFrame(list(collection.find()))



In [0]:
mongo_data

,_id,product_category_name,product_category_name_english
0,683bb4842d2feee59bef2778,beleza_saude,health_beauty
1,683bb4842d2feee59bef2779,informatica_acessorios,computers_accessories
2,683bb4842d2feee59bef277a,automotivo,auto
3,683bb4842d2feee59bef277b,cama_mesa_banho,bed_bath_table
4,683bb4842d2feee59bef277c,moveis_decoracao,furniture_decor
...,...,...,...
66,683bb4842d2feee59bef27ba,flores,flowers
67,683bb4842d2feee59bef27bb,artes_e_artesanato,arts_and_craftmanship
68,683bb4842d2feee59bef27bc,fraldas_higiene,diapers_and_hygiene
69,683bb4842d2feee59bef27bd,fashion_roupa_infanto_juvenil,fashion_childrens_clothes


In [0]:
# display(products_df)


Cleaning the Data

In [0]:
from pyspark.sql.functions import col,to_date,date_diff,current_date

In [0]:
#removing duplicates

def clean_dataframe(df,name):
    print("Cleaning"+name)
    return df.dropDuplicates().na.drop('all')

orders_df=clean_dataframe(orders_df,"orders")
#display(orders_df)
products_df=clean_dataframe(products_df,"products")
#display(products_df)
items_df=clean_dataframe(items_df,"items")
#display(items_df)
sellers_df=clean_dataframe(sellers_df,"sellers")
#display(sellers_df)
payments_df=clean_dataframe(payments_df,"payments")
#display(payments_df)




Cleaningorders
Cleaningproducts
Cleaningitems
Cleaningsellers
Cleaningpayments


In [0]:
orders_df=orders_df.withColumn("order_purchase_timestamp",to_date(col("order_purchase_timestamp")))\
    .withColumn("order_delivered_customer_date",to_date(col("order_delivered_customer_date")))\
    .withColumn("order_estimated_delivery_date",to_date(col("order_estimated_delivery_date")))\
    .withColumn("order_approved_at",to_date(col("order_approved_at")))

In [0]:
#Calculate Delivery and Time Delays
orders_df=orders_df.withColumn("actual_delivery_time",date_diff(col("order_delivered_customer_date"),col("order_purchase_timestamp")))\
    .withColumn("estimates_delivery_time",date_diff(col("order_estimated_delivery_date"),col("order_purchase_timestamp")))\
        .withColumn("delay",col("estimates_delivery_time")>col("actual_delivery_time"))\
        .withColumn("delay_time",col("estimates_delivery_time")-col("actual_delivery_time"))
# display(orders_df)

###Joining

In [0]:
orders_customer_df = orders_df.join(customer_df, "customer_id", "left")

orders_payments_df = orders_customer_df.join(payments_df, "order_id", "left")

orders_items_df = orders_payments_df.join(items_df, "order_id", "left")
orders_items_products_df = orders_items_df.join(products_df, "product_id", "left")
final_df = orders_items_products_df.join(sellers_df, "seller_id", "left") 

In [0]:
# display(final_df)

In [0]:
if '_id' in mongo_data.columns:
    mongo_data.drop('_id', axis=1, inplace=True)


In [0]:
mongo_data.drop('_id', axis=1, inplace=True, errors='ignore')
mongo_spark_df = spark.createDataFrame(mongo_data)
# display(mongo_spark_df)


In [0]:
final_df=final_df.join(mongo_spark_df,"product_category_name","left")
# display(final_df)


What is Parquet?
Apache Parquet is:

A columnar storage format for data (ideal for big data and analytics).
Supported by Spark, Hadoop, Hive, AWS (like S3), and many modern data tools.
Efficient for both storage and query performance:
Compresses well.
Reads only necessary columns during queries.


In [0]:
final_df.columns

['product_category_name',
 'seller_id',
 'product_id',
 'order_id',
 'customer_id',
 'order_status',
 'order_purchase_timestamp',
 'order_approved_at',
 'order_delivered_carrier_date',
 'order_delivered_customer_date',
 'order_estimated_delivery_date',
 'actual_delivery_time',
 'estimates_delivery_time',
 'delay',
 'delay_time',
 'customer_unique_id',
 'customer_zip_code_prefix',
 'customer_city',
 'customer_state',
 'payment_sequential',
 'payment_type',
 'payment_installments',
 'payment_value',
 'order_item_id',
 'shipping_limit_date',
 'price',
 'freight_value',
 'product_name_lenght',
 'product_description_lenght',
 'product_photos_qty',
 'product_weight_g',
 'product_length_cm',
 'product_height_cm',
 'product_width_cm',
 'seller_zip_code_prefix',
 'seller_city',
 'seller_state',
 'product_category_name_english']

In [0]:
final_df.write.mode("overwrite").parquet("abfss://olistdata@olistdatastorageacthyma.dfs.core.windows.net/silver")

In [0]:
from pyspark.sql.functions import sum

# Grouping by product category and summing up total price to get total sales
top_categories = final_df.groupBy("product_category_name_english") \
                         .agg(sum("price").alias("total_sales")) \
                         .orderBy("total_sales", ascending=False) \
                         .limit(10)

# Display the result in a Databricks output cell to enable visualization
# display(top_categories)


In [0]:
# Required imports
from pyspark.sql.functions import sum, avg, count, col, month, when, datediff

In [0]:
# Top Cities by Number of Orders
top_cities = final_df.groupBy("customer_city") \
    .agg(count("order_id").alias("num_orders")) \
    .orderBy("num_orders", ascending=False) \
    .limit(10)
# display(top_cities)

